# Sagemaker con Tensorflow: IMDB.

In [4]:
!pip install -q sagemaker-experiments

You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.


In [3]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'module_4/part_9'

In [2]:
import numpy as np
import os

In [6]:
import tensorflow as tf
from tensorflow.python.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

#### Preparación de los datos
- Base de datos de opiniones de IMDB
- https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data

In [7]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

17465344/17464789 [==============================] - 1s 0us/step
25000 train sequences
25000 test sequences
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [8]:
x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [9]:
data_dir = os.path.join(os.getcwd(), 'imdb_data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'imdb_data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'imdb_data/test')
os.makedirs(test_dir, exist_ok=True)

csv_test_dir = os.path.join(os.getcwd(), 'imdb_data/csv-test')
os.makedirs(csv_test_dir, exist_ok=True)

np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)
np.savetxt(os.path.join(csv_test_dir, 'csv-test.csv'), 
           np.array(x_test[:100], dtype=np.int32), fmt='%d', delimiter=",")

In [10]:
traindata_s3_prefix = f'{prefix}/imdb_data/train'
testdata_s3_prefix = f'{prefix}/imdb_data/test'

train_s3 = sess.upload_data(path='./imdb_data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./imdb_data/test/', key_prefix=testdata_s3_prefix)

## Entrenamiento del Modelo

In [11]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
from time import gmtime, strftime
import time

experiment_name = 'imdb-sentiment-analysis'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training a sentiment classification model using imdb dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')
    

In [12]:
from sagemaker.tensorflow import TensorFlow

exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'imdb-tf-{exp_datetime}'

s3_output_location = f's3://{bucket}/{prefix}/{jobname}'
code_dir = f's3://{bucket}/{prefix}/{jobname}'

train_instance_type = 'ml.p3.2xlarge'
hyperparameters = {'epochs': 10, 'batch_size': 64, 
                   'learning_rate': 0.01, 'drop_out_rate': 0.2}
model_local_path = '/opt/ml/model'

estimator = TensorFlow(source_dir='tf_code',
                       entry_point='tensorflow_sentiment.py',
                       output_path=s3_output_location,
                       code_location=code_dir,
                       model_dir=model_local_path,
                       instance_type=train_instance_type,
                       instance_count=1,
                       enable_sagemaker_metrics=True,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='2.1',
                       py_version='py3')

data_channels = {'train':train_s3, 'test': test_s3}
print(data_channels)

{'train': 's3://sagemaker-eu-west-1-467432373215/module_4/part_9/imdb_data/train', 'test': 's3://sagemaker-eu-west-1-467432373215/module_4/part_9/imdb_data/test'}


In [13]:
# Creating a new trial for the experiment
exp_trial = Trial.create(experiment_name=experiment_name, 
                         trial_name=jobname)

experiment_config={'ExperimentName': experiment_name,
                   'TrialName': exp_trial.trial_name,
                   'TrialComponentDisplayName': 'Training'}

estimator.fit(inputs=data_channels,
              job_name=jobname,
              experiment_config=experiment_config,
              logs=True)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: imdb-tf-2022-12-12-20-40-32


2022-12-12 20:40:39 Starting - Starting the training job...
2022-12-12 20:41:03 Starting - Preparing the instances for trainingProfilerReport-1670877639: InProgress
.........
2022-12-12 20:42:24 Downloading - Downloading input data...
2022-12-12 20:43:02 Training - Downloading the training image.........
2022-12-12 20:44:41 Uploading - Uploading generated training model.2022-12-12 20:44:36,077 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-12-12 20:44:36,350 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch_size": 64,
        "drop_out_rate": 0.2,
        "epochs": 10,
  

UnexpectedStatusException: Error for Training job imdb-tf-2022-12-12-20-40-32: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/local/bin/python3.6 tensorflow_sentiment.py --batch_size 64 --drop_out_rate 0.2 --epochs 10 --learning_rate 0.01 --model_dir /opt/ml/model"
  File "tensorflow_sentiment.py", line 1
    %%writefile code/tensorflow_sentiment.py
    ^
SyntaxError: invalid syntax, exit code: 1

#### Prueba del modelo en local

In [ ]:
!mkdir ./imdb_data/model -p
!aws s3 cp {estimator.model_data} ./imdb_data/model.tar.gz
!tar -xzf ./imdb_data/model.tar.gz -C ./imdb_data/model/

In [ ]:
my_model=tf.keras.models.load_model('./imdb_data/model/1/')

In [ ]:
my_model.summary()

In [ ]:
loss, acc=my_model.evaluate(x_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))